In [25]:
# Import our necessary libraries
import pandas as pd 
from bs4 import BeautifulSoup
import requests

# Set our pandas dataframe width wider
pd.set_option('display.max_colwidth', 400)

In [28]:
def beautify_website():
  """
  returns: BeautifulSoup object
  """
  # Open our scraping website with requests
  url = 'http://quotes.toscrape.com/tableful/'
  website = requests.get(url=url)
  # Grab the soup object from our website markup, and return it
  soup = BeautifulSoup(website.content)
  del website, url
  return soup

def grab_quotes():
  """
  returns: authors, quotes
  """
  # Set our final arrays empty for appending later
  authors = []
  quotes = []
  # Grab out beautified website object
  website = beautify_website()
  # Iterate through all <td> tags with the style:'padding-top: 2em;'
  for tr in website.find_all('td',{'style':'padding-top: 2em;'}):
    # Remove the “ character and split on the ” character.
    # Then grab the quote on the 0 index of the split
    quotes.append(tr.get_text().replace('“','').split('”')[0])
    # Remove the “ character and split on the ” character.
    # Then grab the author on the 1st index of the split, and split it again on ':' (Author:Einstein) and grab the 1st index. 
    # Append to authors.                                                                   ^
    authors.append(tr.get_text().replace('“','').split('”')[1].split(':')[1])
  del website
  return (authors, quotes)

def generate_tags():
  """
  returns: a list of tuples with the original pos, tag and contents of that tag.
  """
  tags = []
  website = beautify_website()
  # Find all tags <td> with the style:'padding-bottom 2em;' in the document
  for td in website.find_all('td',{'style':'padding-bottom: 2em;'},True):
    # Append a list comprehension of the tags and the original position of the tags
    # while leaving out "Tags:" and the newlines
    tags.append([(idx, q) for idx, q in enumerate(set(td.contents)) if q != '\n' if q != 'Tags:\n                \n                '])
  del website
  return tags

def grab_tags():
  """
  returns: a list of each quotes (in a list) tags. (I.e. [['quote1tag1','quote2tag1'],['quote2tag1','quote2tag2']])
  """
  tag = ''
  tags = []
  result = []
  #                                                             length 2                    length 2
  # Iterate over the length of the quotes tags (i.e. [['quote1tag1','quote2tag1'],['quote2tag1','quote2tag2']])
  for i in range(len(generate_tags())):
    # Iterate over each individual tag in the quote (generate_tags()[i])
    for id, q in generate_tags()[i]:
      # Concatenate tag text with a comma, and add it to tag (value,value,value,)
      tag += q.text + ','
    # Append tag (tag now meaning all the tags for the quote, separated by commas) in own list, to group of tags
    tags.append([tag])
    # Reset tag to nothing, for the next group of tags relating to the quote
    tag = ''
  # Iterate through each individual tag using a nested for-loop, and remove the last comma (value,value,value,)
  #                                                                                                          ^
  for q_tag in tags:
    for tag in q_tag:
      # Remove the comma
      result.append(tag[:-1])
  del tag, tags
  return result

def grab_dataframe():
  """
  returns: a pandas dataframe containing all the data from http://quotes.toscrape.com/tableful/
  """
  # Run all functions and return a DataFrame with the lists.
  authors, quotes = grab_quotes()
  tags = grab_tags()
  df = pd.DataFrame({'authors':authors,'quotes':quotes,'tags':tags})
  # Sort the dataframe and reset the index. Drop the original index
  df = df.sort_values('authors').reset_index().drop('index',axis=1)
  return df

In [ ]:
df = grab_dataframe()

In [31]:
df

,authors,quotes,tags
0,Albert Einstein,The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.,"world,change,thinking,deep-thoughts"
1,Albert Einstein,There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.,"inspirational,miracles,live,miracle,life"
2,Albert Einstein,Try not to become a man of success. Rather become a man of value.,"success,value,adulthood"
3,André Gide,It is better to be hated for what you are than to be loved for what you are not.,"love,life"
4,Eleanor Roosevelt,A woman is like a tea bag; you never know how strong it is until it's in hot water.,misattributed-eleanor-roosevelt
5,J.K. Rowling,"It is our choices, Harry, that show what we truly are, far more than our abilities.","choices,abilities"
6,Jane Austen,"The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.","aliteracy,books,classic,humor"
7,Marilyn Monroe,"Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.","inspirational,be-yourself"
8,Steve Martin,"A day without sunshine is like, you know, night.","humor,simile,obvious"
9,Thomas A. Edison,"I have not failed. I've just found 10,000 ways that won't work.","inspirational,failure,paraphrased,edison"
